In [ ]:
import pandas as pd
import numpy as np
import h5py
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import os

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 1000)  # Set max width

# Load spaCy's English model
nlp = spacy.load('en_core_web_sm')

In [ ]:
def preprocess_text(text):
    # Define interrogative words to KEEP
    interrogatives = {"what", "why", "how", "who", "where", "when", "which", "whom", "whose", "no", "not",
                    "very" ,"too" ,"too" ,"just", "if", "but", "however", "without", "like"}
    custom_stopwords = set(nlp.Defaults.stop_words)
    custom_stopwords -= interrogatives

    doc = nlp(text.lower().strip())  # Lowercase and remove whitespace
    
# Process tokens: lemmatize, filter stopwords/punct/numbers, keep interrogatives
    tokens = [
        token.lemma_ 
        for token in doc 
        if (
            (not token.is_stop or token.text in interrogatives) and  # Keep interrogatives
            not token.is_punct and token.is_alpha                                  # Remove punctuation
            # (token.is_alpha or token.like_num)                       # Keep words/numbers
        )
    ]

    return ' '.join(tokens)

In [ ]:
label_mapper = {
    'BT1' : 'knowledge',
    'BT2' : 'comprehension',
    'BT3' : 'application',
    'BT4' : 'analysis',
    'BT5' : 'synthesis',
    'BT6' : 'evaluation'
}

# Load dataset
df = pd.DataFrame()
for i in range(1,5):
    q_df = pd.read_csv(os.getcwd().replace('notebook' , 'dataset') + '/dataset' + str(i) + '.csv')
    df = pd.concat([df , q_df])

# Apply preprocessing
mask = df['label'].isin(label_mapper.keys())
df['label'] = df['label'].mask(mask, df['label'].map(label_mapper))

df['label'] = df['label'].str.lower()

df['processed_question'] = df['question'].apply(preprocess_text)

df['processed_question'] = [''.join(text) for text in df['processed_question']]

In [10]:
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import openai

# Step 1: Input query
query = "Why is the sky blue?"
label = "Application"

# Step 2: Generate synthetic queries
def generate_synthetic_queries(prompt, label, num=5):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": f"Generate {num} alternative or related questions for: {prompt} which statisfy the blooms level of understanding labelled {label}"}
        ]
    )
    return response['choices'][0]['message']['content'].split("\n")

synthetic_queries = generate_synthetic_queries(query, label, 10)

print(synthetic_queries)

# # Step 3: Embed queries
# model = SentenceTransformer('all-MiniLM-L6-v2')
# embeddings = model.encode(synthetic_queries)

# # Step 4: Add to vector store (FAISS example)
# dimension = embeddings.shape[1]
# index = faiss.IndexFlatL2(dimension)
# index.add(embeddings)

# # Search later:
# query_vector = model.encode([query])
# D, I = index.search(query_vector, k=3)


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
